In [14]:
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm
import time
from datetime import datetime

from geopy.geocoders import ArcGIS

import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon

import requests
import re
from bs4 import BeautifulSoup
import json
import urllib

In [2]:
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings("ignore")

geolocator = ArcGIS()

In [ ]:
# https://www.reformagkh.ru/opendata?gid=2208161&page=8&pageSize=10
df = pd.read_csv('./samara_building_ref.csv',sep=';',encoding='utf-8')
df = df.reset_index(drop=True)
df = df.reset_index()

df_gcd = df#[['address','index']]
df_gcd['address'] = 'Россия, Самарская область, ' + df_gcd['address']

In [ ]:
def geocode_address(df,column):
    print(datetime.now())
    tm = round((len(df) / 1.7 / 60), 1)
    print("Estimated time:{} minutes".format(tm))
    df['location'] = df[column].apply(geolocator.geocode, timeout=None)
    df['lat'] = df['location'].apply(lambda x: x.latitude if x!=None else None)
    df['lon'] = df['location'].apply(lambda x: x.longitude if x!=None else None)
    print(datetime.now())
    return df

In [ ]:
def create_points(df):
    np_dn = df.to_numpy()
    ind_lat = list(df.columns).index('lat')
    ind_lon = list(df.columns).index('lon')
    ind_lct = list(df.columns).index('location')

    lst_pnt=[]
    lst_lctn = []
    i=0
    for i in range(len(np_dn)):
        lat = np_dn[i,ind_lat]
        lon = np_dn[i,ind_lon]
        lctn = str(np_dn[i,ind_lct]).replace(')','').replace('(','')
        lst_pnt.append(Point(lon,lat))
        lst_lctn.append(lctn)
    # 

    gdf_pt = gpd.GeoDataFrame(df,geometry=lst_pnt)
    gdf_pt.crs='epsg:4326'
    gdf_pt['location'] = lst_lctn

    return gdf_pt

In [ ]:
gdf_pt = geocode_address(df_gcd,'address')

In [ ]:
gdf_pt = create_points(gdf_pt)

In [ ]:
gdf_pt = gdf_pt[['mun_obr_oktmo',
                     'address',
                    'architectural_monument_status',
                    'total_sq',
                    'total_rooms_amount',
                    'living_rooms_amount',
                    'living_rooms_with_nonresidental_amount',
                    'total_rooms_sq',
                    'living_rooms_sq',
                    'living_rooms_with_nonresidental_sq',
                    'total_ppl',
                    'house_id', 
                     'location','lat', 'lon', 'geometry']]
#

In [ ]:
gdf_pt.to_file('gdf_pt_all2.json',driver='GeoJSON',encoding='utf-8')

In [3]:
gdf_pt = gpd.read_file('new_gdf_13.json',encoding='utf-8')

In [4]:
gdf_poly = gpd.read_file('./borders/outer_border_SO.shp',encoding='utf-8')

In [5]:
gdf_poly.geometry = gdf_poly.geometry.to_crs('epsg:32637').buffer(100).to_crs('epsg:4326')

In [6]:
print(datetime.now())
%time pt_in_brdr = gpd.sjoin(gdf_pt, gdf_poly[['geometry']], how='inner',op='within')

2020-07-30 09:41:01.032664
Wall time: 1.2 s


In [11]:
pt_not_in_brdr = gdf_pt[~gdf_pt.address.isin(pt_in_brdr.address)].reset_index(drop=True)

In [9]:
len(pt_not_in_brdr)

13

In [15]:
def find_js(lnk):
    with requests.get(lnk, stream=True, timeout=25) as req:
        bs = BeautifulSoup(req.text, 'html.parser')
        all_scrpt = bs.find_all("script")
        for scr in all_scrpt:
            if scr.get('type') == "application/json":
                sc_ind = all_scrpt.index(scr)
                break
            else:
                sc_ind=0
        # 
        scrp_txt = all_scrpt[sc_ind].text
        js = json.loads(scrp_txt)

        return js

In [30]:
lst_addr = list(pt_not_in_brdr.address)

lst_lon,lst_lat,lst_lctn,lst_pnt=[],[],[],[]

i=0
for i in tqdm(range(len(lst_addr))):
    addr = 'Россия, Самарская область, ' + lst_addr[i]
    str_qt = urllib.parse.quote(addr)
    srch_ct = "https://yandex.ru/maps/?text={}".format(str_qt)
    try:
        js = find_js(srch_ct)
        location = js['searchPreloadedResults']['items'][0]['address']
        coords = js['searchPreloadedResults']['items'][0]['coordinates']
        lon = coords[0]
        lat = coords[1]
    except:
        print(addr)
        lon,lat,location = 0.0,0.0,0
    lst_lon.append(lon)
    lst_lat.append(lat)
    lst_lctn.append(location)
    lst_pnt.append(Point(lon,lat))
#


In [35]:
new_find = pt_not_in_brdr.copy()
new_find['location'] = lst_lctn
new_find['lat'] = lst_lat
new_find['lon'] = lst_lon
new_find['geometry'] = lst_pnt


In [38]:
gdf_ok = pt_in_brdr.append(new_find).reset_index(drop=True)

In [40]:
gdf_ok.to_file('gdf_ok.json',driver='GeoJSON',encoding='utf-8')